<a href="https://colab.research.google.com/github/gsoraw/Applied-ML-Project-GeorgeSorondo/blob/main/AI24Ch3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# AI Tile Puzzle Comparison Notebook

# --- 📦 Imports and Puzzle Setup ---
import random
import heapq
import pandas as pd

SIZE_3 = 3
SIZE_4 = 4
GOAL_3x3 = list(range(1, 9)) + [0]
GOAL_4x4 = list(range(1, 16)) + [0]
MOVES = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}

def index_to_xy(index, size):
    return divmod(index, size)

def xy_to_index(x, y, size):
    return x * size + y

def is_valid(x, y, size):
    return 0 <= x < size and 0 <= y < size

def get_neighbors(state, size):
    zero_index = state.index(0)
    x, y = index_to_xy(zero_index, size)
    neighbors = []
    for action, (dx, dy) in MOVES.items():
        new_x, new_y = x + dx, y + dy
        if is_valid(new_x, new_y, size):
            new_index = xy_to_index(new_x, new_y, size)
            new_state = state[:]
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            neighbors.append((action, new_state))
    return neighbors

def random_walk(state, steps, size):
    current = state[:]
    for _ in range(steps):
        neighbors = get_neighbors(current, size)
        _, current = random.choice(neighbors)
    return current

def misplaced_tiles(state, goal):
    return sum(1 for i in range(len(state)) if state[i] != 0 and state[i] != goal[i])

def manhattan_distance(state, goal, size):
    distance = 0
    for i, tile in enumerate(state):
        if tile != 0:
            goal_index = goal.index(tile)
            x1, y1 = index_to_xy(i, size)
            x2, y2 = index_to_xy(goal_index, size)
            distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# --- 🔍 BFS and A* Implementation ---
def bfs(start, goal, size):
    queue = [(start, [])]
    visited = set()
    visited.add(tuple(start))
    nodes_expanded = 0
    while queue:
        current, path = queue.pop(0)
        nodes_expanded += 1
        if current == goal:
            return path, nodes_expanded
        for action, neighbor in get_neighbors(current, size):
            state_tuple = tuple(neighbor)
            if state_tuple not in visited:
                visited.add(state_tuple)
                queue.append((neighbor, path + [action]))
    return None, nodes_expanded

def a_star(start, goal, size, heuristic):
    frontier = []
    heapq.heappush(frontier, (0, start, []))
    visited = set()
    visited.add(tuple(start))
    nodes_expanded = 0
    while frontier:
        cost, current, path = heapq.heappop(frontier)
        nodes_expanded += 1
        if current == goal:
            return path, nodes_expanded
        for action, neighbor in get_neighbors(current, size):
            state_tuple = tuple(neighbor)
            if state_tuple not in visited:
                visited.add(state_tuple)
                h = heuristic(neighbor, goal)
                heapq.heappush(frontier, (len(path) + 1 + h, neighbor, path + [action]))
    return None, nodes_expanded

# --- 🎲 Generate Problems ---
def generate_all_problems():
    problems = []
    for size, goal in [(SIZE_3, GOAL_3x3), (SIZE_4, GOAL_4x4)]:
        for steps in [5, 10, 20, 40, 80]:
            for _ in range(3):
                start = random_walk(goal, steps, size)
                problems.append((size, goal, start, steps))
    return problems

# --- 🧪 Run Experiments ---
problems = generate_all_problems()
results = []

for size, goal, start, steps in problems:
    # BFS
    path_bfs, nodes_bfs = bfs(start, goal, size)
    results.append({
        "Size": f"{size}x{size}",
        "Steps": steps,
        "Algorithm": "BFS",
        "Start State": start,
        "Solution Length": len(path_bfs) if path_bfs else -1,
        "Nodes Expanded": nodes_bfs
    })
    # A* with Misplaced Tile
    path_am, nodes_am = a_star(start, goal, size, lambda s, g: misplaced_tiles(s, g))
    results.append({
        "Size": f"{size}x{size}",
        "Steps": steps,
        "Algorithm": "A* Misplaced",
        "Start State": start,
        "Solution Length": len(path_am) if path_am else -1,
        "Nodes Expanded": nodes_am
    })
    # A* with Manhattan Distance
    path_md, nodes_md = a_star(start, goal, size, lambda s, g: manhattan_distance(s, g, size))
    results.append({
        "Size": f"{size}x{size}",
        "Steps": steps,
        "Algorithm": "A* Manhattan",
        "Start State": start,
        "Solution Length": len(path_md) if path_md else -1,
        "Nodes Expanded": nodes_md
    })

# --- 📊 Display Results ---
df_results = pd.DataFrame(results)
df_results
